# Cleaning and preparing of the wine dataset

In [2]:
import pandas as pd

file_path = "../data/raw/winemag-data-130k-v2.csv"

# Loading and cleaning by chunks

In [3]:
chunksize = 20000
clean_chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Renommage des colonnes
    chunk = chunk.rename(columns={
        "Unnamed: 0": "id",
        "region_1": "primary_region",
        "region_2": "secondary_region",
    })

    # Deleting completely empty lines
    chunk = chunk.dropna(how="all")

    # Removing duplicates
    chunk = chunk.drop_duplicates()

    clean_chunks.append(chunk)

# Concaténation de tous les chunks
df_clean = pd.concat(clean_chunks, ignore_index=True)

# Final removal of duplicates across the entire dataset
df_clean = df_clean.drop_duplicates()

print(f"Number of lines after cleaning : {len(df_clean)}")
print(f"\nData overview :")
print(df_clean.head())

Number of lines after cleaning : 129971

Data overview :
   id   country                                        description  \
0   0     Italy  Aromas include tropical fruit, broom, brimston...   
1   1  Portugal  This is ripe and fruity, a wine that is smooth...   
2   2        US  Tart and snappy, the flavors of lime flesh and...   
3   3        US  Pineapple rind, lemon pith and orange blossom ...   
4   4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

        primary_region   secondary_region         taster_name  \
0     

## Deleting columns `secondary_region` and `taster_twitter_handle` and the lines with empty values

In [4]:
if "secondary_region" in df_clean.columns:
    df_clean = df_clean.drop("secondary_region", axis=1)
else:
    print('Column secondary_region doesn\'t exist.')
if "taster_twitter_handle" in df_clean.columns:
    df_clean = df_clean.drop("taster_twitter_handle", axis=1)
else:
    print('Column taster_twitter_handle doesn\'t exist.')


missing_values = df_clean.isnull().sum()

if missing_values.sum() > 0:
    print(f"Before : {len(df_clean)} lines")
    df_clean = df_clean.dropna()
    print(f"After : {len(df_clean)} lines")
    print(f"Deleted lines : {missing_values.sum()}")
else:
    print("No missing values.")

Before : 129971 lines
After : 54170 lines
Deleted lines : 94079


In [5]:
missing_values = df_clean.isnull().sum()
if missing_values.sum() > 0:
    print("\nMissing values per columns :")
    missing_percent = (missing_values / len(df_clean)) * 100
    missing_df = pd.DataFrame({
        'Missing values': missing_values,
        'Percentage': missing_percent
    })
    print(missing_df[missing_df['Missing values'] > 0].sort_values('Percentage', ascending=False))
else:
    print("No missing values.")



No missing values.


# Backup of cleaned Dataset

In [6]:
df_clean.to_csv("../data/cleaned/wine_data_cleaned.csv", index=False)
print(f"\nData saved : {len(df_clean)} lines")



Data saved : 54170 lines
